In [1]:
import bqplot
from bqplot import LinearScale, Lines, Figure, Axis, DateScale, DateColorScale, Scatter, Tooltip, ColorScale,Toolbar
from ipywidgets import Layout, Label
from bqplot.interacts import BrushIntervalSelector, LassoSelector, PanZoom
from traitlets import link
from collections import OrderedDict
import ipywidgets as widgets
import numpy as np
import pandas as pd
import qgrid

periods = 100
security_1 = np.cumsum(np.random.randn(periods)) + 100.
security_2 = security_1 + np.cumsum(np.random.randn(periods))*0.5 
dates = pd.date_range(start='06-01-2007', periods=periods, freq='d')
df = pd.DataFrame(index=dates, data={'security_1':security_1, 'security_2':security_2})
qg = qgrid.show_grid(df)

# Plot and Brush

In [2]:
def plot_and_brush_df(df):
    
    margins = dict(left=25, top=50, bottom=50, right=25)

    x_dt = DateScale()
    scale_y = LinearScale()
    line = Lines(x=dates, y=security_1, scales={'x': x_dt, 'y': scale_y})
    ax_x = Axis(scale=x_dt, grid_lines='solid')
    ax_y = Axis(scale=scale_y, orientation='vertical', tick_format='0.0f')
    brush = BrushIntervalSelector(scale=x_dt)
    fig = Figure(axes=[ax_x, ax_y], marks=[line], fig_margin=margins, interaction=brush)


    # we will share the same y scale
    x_dt2 = DateScale()
    ax_x2 = Axis(scale=x_dt2, grid_lines='solid')
    ax_y2 = Axis(scale=scale_y, orientation='vertical',tick_format='0.0f')
    line2 = Lines(x=[], y=[], scales={'x': x_dt2, 'y': scale_y})
    fig2 = Figure(axes=[ax_x2, ax_y2], marks=[line2], fig_margin=margins)

    def event_handler(name):
        sel = brush.selected
        if sel is None or len(sel) < 2:
            line2.x=[]
            line2.y=[]
        else:
            t_0 = pd.to_datetime(str(sel[0]))
            t_1 = pd.to_datetime(str(sel[1]))
            df_selected = df[t_0:t_1]
            line2.x=df_selected.index
            line2.y=df_selected.values.T[0]

    brush.observe(event_handler)
    return fig, fig2

In [3]:
f1, f2 = plot_and_brush_df(df)

In [4]:
widgets.HBox([f1, f2])

# Plot and Brush Scatter

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [6]:
def plot_and_brush_scatter(df):

    lr = LinearRegression()
    
    margins = dict(left=25, top=50, bottom=50, right=25)

    x_dt = DateScale()
    scale_y = LinearScale()
    line_c = Lines(x=[], y=[], scales={'x': x_dt, 'y': scale_y})
    line_c.opacities = [0.65, 0.90]
    ax_x = Axis(scale=x_dt, grid_lines='solid')
    ax_y = Axis(scale=scale_y, orientation='vertical', tick_format='0.0f')
    brush_s = BrushIntervalSelector(scale=x_dt)
    fig_comp = Figure(axes=[ax_x, ax_y], marks=[line_c], fig_margin=margins, interaction=brush_s)

    color_scale = DateColorScale(scheme='Blues')
    scale_s_x = LinearScale()
    scale_s_y = LinearScale()
    reg_line = Lines(x=[], y=[], scales={'x': scale_s_x, 'y': scale_s_y})

    label = Label()
    def on_hover(w, target):
        label.value = dots.dates[target['data']['index']].strftime("%b-%Y")

    dots = Scatter(x=[], y=[], color=[], scales={'x': scale_s_x, 'y': scale_s_y, 'color': color_scale},
                  tooltip=label, unhovered_style={'opacity':0.40})
    dots.on_hover(on_hover)

    ax_s_x = Axis(scale=scale_s_x, grid_lines='solid')
    ax_s_y = Axis(scale=scale_s_x, orientation='vertical')
    
    fig_s = Figure(axes=[ax_s_x, ax_s_y], marks=[dots, reg_line], fig_margin=margins, animation_duration = 0)

        
    def event_handler(name):
        global dates
        sel = brush_s.selected
        if sel is None or len(sel) < 2:
            dots.x = []
            dots.y = []
            reg_line.x = []
            reg_line.y = []
            fig_s.title = ''
        else:
            t_0 = pd.to_datetime(str(sel[0]))
            t_1 = pd.to_datetime(str(sel[1]))
            df = pd.DataFrame(data={'a':fig_comp.marks[0].y[0], 'b':fig_comp.marks[0].y[1]}, index=fig_comp.marks[0].x)
            df_selected = df[t_0:t_1]
            dots.x = df_selected.values[:,0]
            dots.y = df_selected.values[:,1]
            dots.dates = df_selected.index   # you can stick anything into dots (?)
            #dots.color = df_selected.index
            fig_s.title = fig_comp.title
            try:
                lr.fit(dots.x[:, np.newaxis], dots.y)
                with reg_line.hold_sync():
                    # clear them first so the line animation isn't janky
                    y = lr.predict(dots.x[:, np.newaxis])
                    r2 = r2_score(dots.y, y)
                    reg_line.opacities = [max(r2, 0.15)]
                    reg_line.x = []
                    reg_line.y = []            
                    reg_line.x = dots.x
                    reg_line.y = y
            except:
                reg_line.x = []
                reg_line.y = []

    brush_s.observe(event_handler)
    
    fig_comp.marks[0].y = [df.values[:,0], df.values[:,1]] # joint.values[:,1] #
    fig_comp.marks[0].x = df.index.values
    
    return fig_comp, fig_s, brush_s

In [7]:
f3, f4, brush = plot_and_brush_scatter(df)

In [8]:
widgets.HBox([f3, f4])

# Scatter Lasso


In [83]:
sc_x, sc_y = LinearScale(), LinearScale()
sc_c1 = ColorScale()
sc_size = LinearScale()
sc_opacity = LinearScale()

sc_x_pz = LinearScale()
sc_y_pz = LinearScale()

colors = np.random.randn(len(df))
sizes = np.random.rand(len(df))

scatter = Scatter(
    x=df['security_1'],
    y=df['security_2'],
    scales={'x': sc_x,
            'y': sc_y,
            'color': sc_c1,
            'size': sc_size,
            'opacity': sc_opacity},
    color=colors,
    size=sizes,
    default_size=128)
scatter.default_opacities = [0.75]
scatter.opacity = df.index.values

In [84]:
sc_c1.colors = ['blue', 'green', 'yellow']

In [85]:
ls = LassoSelector()
ls.marks = [scatter]

In [86]:
ax_x = Axis(scale=sc_x, grid_lines='none', visible=False)
ax_y = Axis(scale=sc_y, grid_lines='none', orientation='vertical', visible=False)

In [87]:
margins = dict(left=2, top=2, bottom=2, right=2)
fig = Figure(marks=[scatter],  axes=[ax_x, ax_y],  fig_margin=margins)

#fig.layout.width = '95%'
#fig.layout.height = 'auto'
buttonWidth = '50px'

pz = PanZoom(scales={'x': [sc_x], 'y': [sc_y]})

zoom_interacts = widgets.ToggleButtons(
    options=OrderedDict([
        ('xy ', pz),
        ('a ', ls),   
        (' ', None)]),
    icons = ["arrows", "pencil", "stop"],
    tooltips = ["zoom/pan in x & y", "select", "cancel zoom/pan"]
)

zoom_interacts.style.button_width = buttonWidth

ResetZoomButton = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reset zoom',
    icon='arrows-alt'
)

def resetZoom(new):
    fig.interaction = None  # clear any lasso
    fig.animation_duration = 1000
    with fig.hold_sync():
    # Reset the x and y axes on the figure
        fig.axes[0].scale.min = None
        fig.axes[1].scale.min = None
        fig.axes[0].scale.max = None
        fig.axes[1].scale.max = None
    fig.animation_duration = 0

    
ResetZoomButton.on_click(resetZoom)
ResetZoomButton.layout.width = buttonWidth

link((zoom_interacts, 'value'), (fig, 'interaction'))


In [88]:
# Creating the figure to be displayed as the tooltip
def make_tooltip_box():
    fig_layout = Layout(width='95%', height='200px')
    margins = dict(left=2, top=2, bottom=2, right=2)
    
    sc_x_dt = DateScale()
    sc_y = LinearScale()
    ax_x = Axis(scale=sc_x_dt, visible=False)
    ax_y = Axis(scale=sc_y, orientation='vertical', visible=False)

    line_a = Lines(x=dates, y=[], scales={'x': sc_x_dt, 'y': sc_y}, colors=['blue'])
    fig_line_a = Figure(marks=[line_a], axes=[ax_x, ax_y], layout=fig_layout, fig_margin=margins, animation_duration=1000)
    
    line_b = Lines(x=dates, y=[], scales={'x': sc_x_dt, 'y': sc_y}, colors=['green'])
    fig_line_b = Figure(marks=[line_b], axes=[ax_x, ax_y], layout=fig_layout, fig_margin=margins, animation_duration=1000)

    fig_tooltip = widgets.VBox([fig_line_a, fig_line_b], layout=Layout(width='220px', height='400px'))
    
    return fig_line_a, fig_line_b, fig_tooltip

In [89]:
fig_line_a, fig_line_b, fig_tooltip = make_tooltip_box()

In [90]:
def on_hover(w, target):
    selected_date_index = target['data']['index']
    start_idx = max(0, selected_date_index-20)
    end_idx = start_idx + 60
    
    with fig_line_a.hold_sync():
        fig_line_a.marks[0].y = df.values[start_idx:end_idx, 0]
        fig_line_a.marks[0].x = df.index.values
    
    with fig_line_b.hold_sync():
        fig_line_b.marks[0].y = df.values[start_idx:end_idx, 1]
        fig_line_b.marks[0].x = df.index.values

scatter.tooltip = None
scatter.on_hover(on_hover)

In [97]:
widgets.VBox([
        widgets.HBox([zoom_interacts,ResetZoomButton]),
        widgets.HBox([fig, fig_tooltip])
])

In [19]:
scatter.opacity = df.index.values

In [20]:
df.iloc[scatter.selected, :]

,security_1,security_2


# Graph Lasso

In [21]:
# from bqplot.marks import Graph
# fig_layout = Layout(width='960px', height='800px')

In [22]:
# lasso_sel = LassoSelector()

In [23]:
# node_data = list('ABCDEFG')

# link_matrix = np.zeros((7, 7))
# xs = LinearScale()
# ys = LinearScale()
# x = [100, 200, 200, 300, 300, 300, 300]
# y = [2, .5, 4, 8, 6, 4, 1]
# graph = Graph(node_data=node_data, link_matrix=link_matrix, link_type='line', 
#               colors=['orange'] * 7,
#               scales={'x': xs, 'y': ys, }, x=x, y=y, 
#               directed=False)
# fig = Figure(marks=[graph], layout=fig_layout, interaction=lasso_sel)
# lasso_sel.marks = [graph]

In [24]:
#fig

In [25]:
# graph.selected

In [26]:
# from bqplot.interacts import PanZoom
# import ipywidgets as widgets
# import bqplot as bq
# from traitlets import link
# from collections import OrderedDict

# import numpy as np

# buttonWidth = '50px'

# x_sc = bq.LinearScale()
# y_sc = bq.LinearScale()

# x_data = np.arange(500)
# y_data = np.random.randn(3, 500)

# line_chart = bq.Lines(x=x_data, y=y_data, scales= {'x': x_sc, 'y': y_sc}, 
#                        display_legend=True, labels=["line 1", "line 2", "line 3"] )

# ax_x = bq.Axis(scale=x_sc)
# ax_y = bq.Axis(scale=y_sc, orientation='vertical', tick_format='0.2f')

# fig = bq.Figure(marks=[line_chart], axes=[ax_x, ax_y])
# fig.layout.width = '95%'

# pz = PanZoom(scales={'x': [x_sc], 'y': [y_sc]})
# pzx = PanZoom(scales={'x': [x_sc]})
# pzy = PanZoom(scales={'y': [y_sc], })

# #
# zoom_interacts = widgets.ToggleButtons(
#                                         options=OrderedDict([
#                                             ('xy ', pz), 
#                                             ('x ', pzx), 
#                                             ('y ', pzy),   
#                                             (' ', None)]),
#                                             icons = ["arrows", "arrows-h", "arrows-v", "pencil"],
#                                             tooltips = ["zoom/pan in x & y", "zoom/pan in x only", "zoom/pan in y only", "cancel zoom/pan"]
#                                         )
# zoom_interacts.style.button_width = buttonWidth

# ResetZoomButton = widgets.Button(
#     description='',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Reset zoom',
#     icon='arrows-alt'
# )

# def resetZoom(new):
#     # Reset the x and y axes on the figure
#     fig.axes[0].scale.min = None
#     fig.axes[1].scale.min = None
#     fig.axes[0].scale.max = None
#     fig.axes[1].scale.max = None  
    
# ResetZoomButton.on_click(resetZoom)
# ResetZoomButton.layout.width = buttonWidth

# link((zoom_interacts, 'value'), (fig, 'interaction'))
# widgets.VBox([fig, widgets.HBox([zoom_interacts,ResetZoomButton])], align_self='stretch')